In [ ]:
import pandas as pd
import numpy as np
import json
from urllib.request import urlopen
from pyspark.sql.session import SparkSession
from pyspark import SparkFiles
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql.functions import col, array_contains

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.uiWebUrl

In [ ]:
import urllib.request, json

url5min = 'https://data.sensor.community/static/v2/data.json'
url24h = 'https://data.sensor.community/static/v2/data.24h.json'

# with urllib.request.urlopen(url) as url:
#     data = json.load(url)
# with urllib.request.urlopen(url24h) as url:
#     data24h = json.load(url)
# with open('data/data.json', 'w') as outfile:
#     json.dump(data, outfile)
# with open('data/data24h.json', 'w') as outfile:
#     json.dump(data24h, outfile)

# Save csv to array
air = np.genfromtxt('data/air.csv', delimiter=',', dtype=None, encoding=None)

In [ ]:
url5min = 'https://data.sensor.community/static/v2/data.json'
url24h = 'https://data.sensor.community/static/v2/data.24h.json'

spark.sparkContext.addFile(url24h)
filename = SparkFiles.get(url24h.split('/')[-1])
df24h = spark.read.json(filename)
df24h.createOrReplaceTempView("sensors24h")
df24h.printSchema()

spark.sparkContext.addFile(url5min)
filename = SparkFiles.get(url5min.split('/')[-1])
df5min = spark.read.json(filename)
df5min.createOrReplaceTempView("sensors5min")
df5min.printSchema()

In [ ]:
df = df24h

In [ ]:
# Remove columns that do not contains neither PM10 nor PM2.5 :
df = df.\
    where(array_contains(col('sensordatavalues.value_type'), 'P1')
          | array_contains(col('sensordatavalues.value_type'),'P2'))

In [ ]:
# Remove everything but P1 and P2 in df, by creating a new df using pandas and exploding the sensordatavalues column:
df.select('id', 'sensordatavalues.value_type', 'sensordatavalues.value').show(10, False)    # BEFORE

pdf = df.toPandas()
pdf = pdf.explode('sensordatavalues')
pdf = pdf[pdf['sensordatavalues'].apply(lambda x: x['value_type'] in ['P1', 'P2'])]
pdf = pdf.groupby('id').agg({'sensordatavalues': lambda x: list(x)})
pdf['id'] = pdf.index

df2 = spark.createDataFrame(pdf)

df = df.drop('sensordatavalues')
df = df.join(df2, on='id')

df.select('id', 'sensordatavalues.value_type', 'sensordatavalues.value').show(10, False)    # AFTER

In [ ]:
spark.stop()